In [ ]:
from sklearn.model_selection import train_test_split
import gensim
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, CategoricalNB, ComplementNB
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import hamming_loss

import ast

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

#Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_preprocess.csv')
dataword2vec = df["review_content"]
df.head()

,Unnamed: 0,review_content,review_type,rev_clean,rev_token,rev_removesw,rev_stem
0,145,as a former middle school teacher of gifted li...,0,as a former middle school teacher of gifted li...,"['as', 'a', 'former', 'middle', 'school', 'tea...","['middle', 'school', 'teacher', 'gifted', 'lit...","['middl', 'school', 'teacher', 'gift', 'litera..."
1,146,the lightning thief is an admirable kid's fant...,0,the lightning thief is an admirable kid s fant...,"['the', 'lightning', 'thief', 'is', 'an', 'adm...","['lightning', 'thief', 'admirable', 'kid', 'fa...","['lightn', 'thief', 'admir', 'kid', 'fantasi']"
2,147,"ok, but only just.",1,ok but only just,"['ok', 'but', 'only', 'just']",[],[]
3,148,there's nothing resembling a spark in this fil...,1,there s nothing resembling a spark in this fil...,"['there', 's', 'nothing', 'resembling', 'a', '...","['resembling', 'spark', 'film']","['resembl', 'spark', 'film']"
4,289,please give is truly a film for our age; it's ...,0,please give is truly a film for our age it s d...,"['please', 'give', 'is', 'truly', 'a', 'film',...","['film', 'age', 'darkly', 'comic', 'stuff', 'm...","['film', 'age', 'darkli', 'comic', 'stuff', 'm..."


In [ ]:
import seaborn as sns
import plotly.express as px
fig = px.bar(df.groupby('review_type').size().reset_index(name='count'), 
             x='review_type', y='count', width=480, height=480, 
             barmode='group', title='Distribusi label')
fig.show(renderer='colab')

In [ ]:
X = df[['review_content','rev_stem']]
y = df['review_type']

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
cc = RandomUnderSampler()
X_resampled,y_resampled= cc.fit_resample(X,y)

In [ ]:
from collections import Counter

In [ ]:
print(sorted(Counter(y_resampled).items()))

[(0, 11756), (1, 11756)]


In [ ]:
unique, counts = np.unique(y_resampled, return_counts=True)

print(unique,counts)

[0 1] [11756 11756]


In [ ]:
df_resample = pd.DataFrame(X_resampled)
df_resample = df_resample.join(y_resampled)

In [ ]:
df_resample.head()

,review_content,rev_stem,review_type
0,barantini's villain is a genre film but a rath...,"['barantini', 'villain', 'genr', 'film', 'supe...",0
1,a romantic musical that does not always hit th...,"['romant', 'music', 'hit', 'note', 'miss', 'op...",0
2,"james franco is so deansian, it's scary. the l...","['jame', 'franco', 'deansian', 'scari', 'life'...",0
3,there's an excitement to seeing this landscape...,"['excit', 'landscap', 'yearn', 'hunger', 'cycl...",0
4,"good, but should have been better...",['good'],0


In [ ]:
import seaborn as sns
import plotly.express as px
fig = px.bar(df_resample.groupby('review_type').size().reset_index(name='count'), 
             x='review_type', y='count', width=480, height=480, 
             barmode='group', title='Distribusi label')
fig.show(renderer='colab')

In [ ]:
#df_resample.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_resample.csv')

In [ ]:
df = pd.read_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_resample.csv')
df.head()

,Unnamed: 0,review_content,rev_stem,review_type
0,0,barantini's villain is a genre film but a rath...,"['barantini', 'villain', 'genr', 'film', 'supe...",0
1,1,a romantic musical that does not always hit th...,"['romant', 'music', 'hit', 'note', 'miss', 'op...",0
2,2,"james franco is so deansian, it's scary. the l...","['jame', 'franco', 'deansian', 'scari', 'life'...",0
3,3,there's an excitement to seeing this landscape...,"['excit', 'landscap', 'yearn', 'hunger', 'cycl...",0
4,4,"good, but should have been better...",['good'],0


In [ ]:
print('Sentiment : ', df.groupby(['review_type'])['review_type'].count())

Sentiment :  review_type
0    11756
1    11754
Name: review_type, dtype: int64


In [ ]:
#masukkan kata ke dalam list soalnya kalo ga pake ini, dia datanya perkata untuk di pake data LDA
def convert_text_list(texts): 
    texts = ast.literal_eval(texts)
    return [text for text in texts]
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [ ]:
df["rev_list"] = df['rev_stem'].apply(convert_text_list) #with stemming
df["rev_kalimat"] = df["rev_list"].apply(listToString)

In [ ]:
df.head()

,Unnamed: 0,review_content,rev_stem,review_type,rev_list,rev_kalimat
0,0,barantini's villain is a genre film but a rath...,"['barantini', 'villain', 'genr', 'film', 'supe...",0,"[barantini, villain, genr, film, superior, man...",barantini villain genr film superior manipul s...
1,1,a romantic musical that does not always hit th...,"['romant', 'music', 'hit', 'note', 'miss', 'op...",0,"[romant, music, hit, note, miss, opportun]",romant music hit note miss opportun
2,2,"james franco is so deansian, it's scary. the l...","['jame', 'franco', 'deansian', 'scari', 'life'...",0,"[jame, franco, deansian, scari, life, stori, r...",jame franco deansian scari life stori rebel sh...
3,3,there's an excitement to seeing this landscape...,"['excit', 'landscap', 'yearn', 'hunger', 'cycl...",0,"[excit, landscap, yearn, hunger, cyclic, rage,...",excit landscap yearn hunger cyclic rage treat ...
4,4,"good, but should have been better...",['good'],0,[good],good


In [ ]:
df.isna().sum()

Unnamed: 0        0
review_content    2
rev_stem          0
review_type       0
rev_list          0
rev_kalimat       0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

Unnamed: 0        0
review_content    0
rev_stem          0
review_type       0
rev_list          0
rev_kalimat       0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23510 entries, 0 to 23511
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      23510 non-null  int64 
 1   review_content  23510 non-null  object
 2   rev_stem        23510 non-null  object
 3   review_type     23510 non-null  int64 
 4   rev_list        23510 non-null  object
 5   rev_kalimat     23510 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.3+ MB


Split data

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

def splitData(df):
    X_train, X_test, y_train, y_test = train_test_split(df[['review_content','rev_list','rev_kalimat']], df['review_type'], test_size = 0.3, random_state= 42, shuffle = True)

    df_train = pd.DataFrame(y_train)  
    df_train = df_train.join(X_train)

    df_test = pd.DataFrame(y_test)
    df_test = df_test.join(X_test)

    return df_train, df_test

In [ ]:
data_train, data_test = splitData(df)

In [ ]:
data_train

,review_type,review_content,rev_list,rev_kalimat
21091,1,the biggest problem is the jumbled and hackney...,"[biggest, problem, jumbl, hackney, screenplay,...",biggest problem jumbl hackney screenplay regar...
18183,1,the film tries to cover the logic gaps with co...,"[film, cover, logic, gap, comedi, joke, bad, w...",film cover logic gap comedi joke bad worsen ex...
3795,0,this is a sublime romantic comedy that spins o...,"[sublim, romant, comedi, spin, wilder, favouri...",sublim romant comedi spin wilder favourit them...
22590,1,"unfortunately, it all builds to a last reel th...","[build, reel, rail, strenuou, bit, action, fli...",build reel rail strenuou bit action flick sill...
1280,0,"sombre and sensitive, hittman's film offers a ...","[sombr, sensit, hittman, film, offer, sear, ou...",sombr sensit hittman film offer sear outrag in...
...,...,...,...,...
11964,1,...an absolutely interminable piece of work...,"[absolut, intermin, piec, work]",absolut intermin piec work
21577,1,with a promising title taken from edgar allan ...,"[promis, titl, edgar, allan, poe, poem, haunt,...",promis titl edgar allan poe poem haunt palac m...
5390,0,amusing and unusual film about the bond betwee...,"[amus, unusu, film, bond, titl, charact]",amus unusu film bond titl charact
860,0,uncut gems is driven by a solid performance by...,"[uncut, gem, driven, solid, perform, adam, san...",uncut gem driven solid perform adam sandler en...


In [ ]:
data_train.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train.csv')

In [ ]:
data_test.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test.csv')

In [ ]:
print('Jumlah Data :', len(data_train))
print('Sentiment : ', data_train.groupby(['review_type'])['review_type'].count())

Jumlah Data : 16457
Sentiment :  review_type
0    8228
1    8229
Name: review_type, dtype: int64


In [ ]:
print('Jumlah Data :', len(data_test))
print('Sentiment : ', data_test.groupby(['review_type'])['review_type'].count())

Jumlah Data : 7053
Sentiment :  review_type
0    3528
1    3525
Name: review_type, dtype: int64


#Word2Vec

##Train

In [ ]:
import pandas as pd
import numpy as np

# w2v
import joblib
import ast
import nltk
nltk.download('punkt')
import gensim
from gensim.models import Word2Vec 
import itertools
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
Corpus_voc = [] 
temp = []
# iterate through each sentence in the file 
for i in dataword2vec: 
  temp = [] 
  # tokenize the sentence into words 
  for j in word_tokenize(str(i)): 
      temp.append(j.lower()) 
  Corpus_voc.append(temp)

In [ ]:
data_train['rev_list'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         data_train.rev_list))  # tokenize sentences

In [ ]:
modelskip = Word2Vec(Corpus_voc, min_count = 1 , alpha = 0.1 , size=300, window=5, sg=1)

In [ ]:
joblib.dump(modelskip, '/content/drive/My Drive/TUGAS AKHIR/TA/Code/word2vecskip.jl')
skipgram = joblib.load('/content/drive/My Drive/TUGAS AKHIR/TA/Code/word2vecskip.jl')

In [ ]:
def get_w2v_features(w2v_model, sentence_group):
    words = np.concatenate(sentence_group)  # words in text
    index2word_set = set(w2v_model.wv.vocab.keys())  # words known to model
    
    featureVec = np.zeros(w2v_model.vector_size, dtype="float32")
    
    # Initialize a counter for number of words in a review
    nwords = 0
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

#mengambil arti dari semua vektor kata yang ada pada kalimat untuk mendapatkan representasi vektor 

In [ ]:
data_train

,review_type,review_content,rev_list,rev_kalimat
14475,1,there is quite a bit to dark places that appea...,"[bit, dark, place, appear, market, surfac]",bit dark place appear market surfac
18185,1,the film tries to cover the logic gaps with co...,"[film, cover, logic, gap, comedi, joke, bad, w...",film cover logic gap comedi joke bad worsen ex...
3795,0,scorsese never lets us forget he's an outsider...,"[scorses, forget, outsid, tipto, materi, afrai...",scorses forget outsid tipto materi afraid bump...
14280,1,what starts out as a promising comic thriller ...,"[start, promis, comic, thriller, deflat, clear...",start promis comic thriller deflat clear gore
1280,0,the stakes are low and there's just enough sla...,"[stake, low, slapstick, tomfooleri, thing, ent...",stake low slapstick tomfooleri thing entertain
...,...,...,...,...
11965,1,the ten commandments may have been impressive ...,"[command, impress, time, film, horribl, fail, ...",command impress time film horribl fail attempt...
21578,1,"not as weird or fun as it sounds, the delibera...","[weird, fun, sound, deliber, pace, film, dabbl...",weird fun sound deliber pace film dabbl atmosp...
5391,0,rian johnson kept the mood of the movie rather...,"[rian, johnson, mood, movi, light, seemingli, ...",rian johnson mood movi light seemingli frivol ...
860,0,high life is a story so simple it loops back t...,"[high, life, stori, simpl, loop, profound, foc...",high life stori simpl loop profound focus char...


In [ ]:
# memberi nomor index
index_kosong = []
for i in range(len(data_train.rev_list)):
  if len(data_train.rev_list.iloc[i]) == 0:
    index_kosong.append(i)

In [ ]:
# drop index yang masih kosong 
data_train = data_train.drop(data_train.index[index_kosong])

In [ ]:
data_train

,review_type,review_content,rev_list,rev_kalimat
21091,1,the biggest problem is the jumbled and hackney...,"[[biggest], [problem], [jumbl], [hackney], [sc...",biggest problem jumbl hackney screenplay regar...
18183,1,the film tries to cover the logic gaps with co...,"[[film], [cover], [logic], [gap], [comedi], [j...",film cover logic gap comedi joke bad worsen ex...
3795,0,this is a sublime romantic comedy that spins o...,"[[sublim], [romant], [comedi], [spin], [wilder...",sublim romant comedi spin wilder favourit them...
22590,1,"unfortunately, it all builds to a last reel th...","[[build], [reel], [rail], [strenuou], [bit], [...",build reel rail strenuou bit action flick sill...
1280,0,"sombre and sensitive, hittman's film offers a ...","[[sombr], [sensit], [hittman], [film], [offer]...",sombr sensit hittman film offer sear outrag in...
...,...,...,...,...
11964,1,...an absolutely interminable piece of work...,"[[absolut], [intermin], [piec], [work]]",absolut intermin piec work
21577,1,with a promising title taken from edgar allan ...,"[[promis], [titl], [edgar], [allan], [poe], [p...",promis titl edgar allan poe poem haunt palac m...
5390,0,amusing and unusual film about the bond betwee...,"[[amus], [unusu], [film], [bond], [titl], [cha...",amus unusu film bond titl charact
860,0,uncut gems is driven by a solid performance by...,"[[uncut], [gem], [driven], [solid], [perform],...",uncut gem driven solid perform adam sandler en...


In [ ]:
data_train['w2v_features_skip'] = list(map(lambda sen_group:
                                      get_w2v_features(modelskip, sen_group),
                                      data_train.rev_list))

In [ ]:
feature = [x for x in data_train['w2v_features_skip'].transpose()]
word2vec_with_stop_features_train = np.asarray(feature)

print(word2vec_with_stop_features_train.shape)

(16441, 300)


In [ ]:
X_train = word2vec_with_stop_features_train

In [ ]:
data_train

,review_type,review_content,rev_list,rev_kalimat,w2v_features_skip
21091,1,the biggest problem is the jumbled and hackney...,"[[biggest], [problem], [jumbl], [hackney], [sc...",biggest problem jumbl hackney screenplay regar...,"[0.07902622, -0.11094307, -0.016993998, -0.175..."
18183,1,the film tries to cover the logic gaps with co...,"[[film], [cover], [logic], [gap], [comedi], [j...",film cover logic gap comedi joke bad worsen ex...,"[0.052281983, 0.011325347, -0.2667158, 0.00600..."
3795,0,this is a sublime romantic comedy that spins o...,"[[sublim], [romant], [comedi], [spin], [wilder...",sublim romant comedi spin wilder favourit them...,"[0.0054527204, -0.064933434, -0.15791309, 0.06..."
22590,1,"unfortunately, it all builds to a last reel th...","[[build], [reel], [rail], [strenuou], [bit], [...",build reel rail strenuou bit action flick sill...,"[0.0742638, -0.07038974, -0.06606511, 0.015170..."
1280,0,"sombre and sensitive, hittman's film offers a ...","[[sombr], [sensit], [hittman], [film], [offer]...",sombr sensit hittman film offer sear outrag in...,"[-0.028482953, 0.020645753, -0.1147002, 0.0677..."
...,...,...,...,...,...
11964,1,...an absolutely interminable piece of work...,"[[absolut], [intermin], [piec], [work]]",absolut intermin piec work,"[-0.1621368, -0.3000759, -0.08028856, 0.312388..."
21577,1,with a promising title taken from edgar allan ...,"[[promis], [titl], [edgar], [allan], [poe], [p...",promis titl edgar allan poe poem haunt palac m...,"[0.02332063, 0.0002150517, -0.10808051, -0.091..."
5390,0,amusing and unusual film about the bond betwee...,"[[amus], [unusu], [film], [bond], [titl], [cha...",amus unusu film bond titl charact,"[0.115980774, 0.054358747, -0.16859347, -0.001..."
860,0,uncut gems is driven by a solid performance by...,"[[uncut], [gem], [driven], [solid], [perform],...",uncut gem driven solid perform adam sandler en...,"[0.047598656, -0.07960564, -0.06566135, -0.022..."


##Test

In [ ]:
data_test['rev_list'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         data_test.rev_list))  # tokenize sentences

In [ ]:
# memberi nomor index
index_kosong = []
for i in range(len(data_test.rev_list)):
  if len(data_test.rev_list.iloc[i]) == 0:
    index_kosong.append(i)

In [ ]:
# drop index yang masih kosong 
data_test = data_test.drop(data_test.index[index_kosong])

In [ ]:
data_test['w2v_features_skip'] = list(map(lambda sen_group:
                                      get_w2v_features(modelskip, sen_group),
                                      data_test.rev_list))

In [ ]:
feature = [x for x in data_test['w2v_features_skip'].transpose()]
word2vec_with_stop_features_test = np.asarray(feature)

print(word2vec_with_stop_features_test.shape)

(7048, 300)


In [ ]:
X_test = word2vec_with_stop_features_test

In [ ]:
data_test

,review_type,review_content,rev_list,rev_kalimat,w2v_features_skip
9658,0,widows is an instant crime classic.,"[[widow], [instant], [crime], [classic]]",widow instant crime classic,"[-0.02011139, 0.16980423, -0.21588689, -0.0757..."
9946,0,"if the next bad boys flicks can be as smart, f...","[[bad], [boy], [flick], [smart], [funni], [emo...",bad boy flick smart funni emot happi wait year,"[-0.032406345, 0.024870163, -0.10458815, 0.045..."
17212,1,poor amy adams.,"[[poor], [ami], [adam]]",poor ami adam,"[0.1377494, -0.0791139, -0.13724047, -0.135527..."
8886,0,i found it really campy and silly and fun.,"[[campi], [silli], [fun]]",campi silli fun,"[-0.2516516, 0.27788475, -0.2466187, 0.2294941..."
18356,1,"it mostly juggles its inspirations instead, an...","[[juggl], [inspir], [central], [conceit], [tra...",juggl inspir central conceit trashili fun comp...,"[-0.11715224, -0.052920707, -0.124827966, 0.07..."
...,...,...,...,...,...
15026,1,"throughout his career as an actor, elvis presl...","[[career], [actor], [elvi], [presley], [involv...",career actor elvi presley involv cinemat atroc...,"[-0.043926843, -0.042537432, -0.060564943, -0...."
4245,0,"""blue story"" is very much a blast of something...","[[blue], [stori], [blast], [tens], [rapman], [...",blue stori blast tens rapman scene boil life c...,"[-0.0048245043, -0.021238593, -0.19684377, -0...."
2310,0,"beautifully storyboarded, each frame looks lik...","[[beauti], [storyboard], [frame], [illustr], [...",beauti storyboard frame illustr written childr...,"[0.037959952, -0.06477052, 0.11009729, -0.1421..."
16596,1,even by the relatively low bar of the previous...,"[[low], [bar], [previou], [film], [fall], [sho...",low bar previou film fall short guess book cas...,"[-0.076478794, 0.0073017827, -0.14866151, -0.0..."


#NB

##Gaussian

In [ ]:
y_train = data_train['review_type']
y_test = data_test['review_type']

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)
pred = model.predict(X_train) #prediksi train
Y_pred = model.predict(X_test) #prediksi test
Y_pred


array([0, 1, 1, ..., 0, 0, 0])

### Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test, Y_pred)*100, "%")
print("Precision: ", precision_score(y_test, Y_pred)*100, "%")
print("Recall: ", recall_score(y_test, Y_pred)*100, "%")
print("F1 Score: ", f1_score(y_test, Y_pred)*100, "%")

Accuracy: 60.30079455164585 %
Precision:  64.22126324048646 %
Recall:  46.466080045415836 %
F1 Score:  53.91963109354414 %


In [ ]:
print("Confusion Matrix")
print(confusion_matrix(y_test, Y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, Y_pred).ravel()
print('TN: ', tn)
print('FP: ', fp)
print('FN: ', fn)
print('TP: ', tp)

Confusion Matrix
[[2613  912]
 [1886 1637]]
TN:  2613
FP:  912
FN:  1886
TP:  1637


In [ ]:
print("Classification report")
print(classification_report(y_test, Y_pred))

Classification report
              precision    recall  f1-score   support

           0       0.58      0.74      0.65      3525
           1       0.64      0.46      0.54      3523

    accuracy                           0.60      7048
   macro avg       0.61      0.60      0.60      7048
weighted avg       0.61      0.60      0.60      7048



### Export data

In [ ]:
data_train_gaus = data_train.assign(sentiment_pred = pred)

In [ ]:
data_train_gaus.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_gaussian_300_pred.csv')

In [ ]:
data_test_gaus = data_test.assign(sentiment_pred = Y_pred)

In [ ]:
data_test_gaus.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_gaussian_300_pred.csv')

##Multinomial

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_multinomial = scaler.fit_transform(X_train)
X_test_multinomial = scaler.fit_transform(X_test)

In [ ]:
model_multinomial = MultinomialNB()
model_multinomial.fit(X_train_multinomial, y_train)
pred_multinomial = model_multinomial.predict(X_train_multinomial) #prediksi train
Y_pred_multinomial = model_multinomial.predict(X_test_multinomial) #prediksi test
Y_pred_multinomial

array([0, 1, 1, ..., 0, 1, 0])

### Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test, Y_pred_multinomial)*100, "%")
print("Precision: ", precision_score(y_test, Y_pred_multinomial)*100, "%")
print("Recall: ", recall_score(y_test, Y_pred_multinomial)*100, "%")
print("F1 Score: ", f1_score(y_test, Y_pred_multinomial)*100, "%")

Accuracy: 64.48637911464246 %
Precision:  75.57673019057172 %
Recall:  42.776043145046835 %
F1 Score:  54.63114011237992 %


In [ ]:
print("Confusion Matrix")
print(confusion_matrix(y_test, Y_pred_multinomial))
tn, fp, fn, tp = confusion_matrix(y_test, Y_pred_multinomial).ravel()
print('TN: ', tn)
print('FP: ', fp)
print('FN: ', fn)
print('TP: ', tp)

Confusion Matrix
[[3038  487]
 [2016 1507]]
TN:  3038
FP:  487
FN:  2016
TP:  1507


In [ ]:
print("Classification report")
print(classification_report(y_test, Y_pred_multinomial))

Classification report
              precision    recall  f1-score   support

           0       0.60      0.86      0.71      3525
           1       0.76      0.43      0.55      3523

    accuracy                           0.64      7048
   macro avg       0.68      0.64      0.63      7048
weighted avg       0.68      0.64      0.63      7048



### Export data

In [ ]:
data_train_multi = data_train.assign(sentiment_pred = pred_multinomial)

In [ ]:
data_train_multi.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_multinomial_300_pred.csv')

In [ ]:
data_test_multi = data_test.assign(sentiment_pred = Y_pred_multinomial)

In [ ]:
data_test_multi.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_multinomial_300_pred.csv')

#Dimensi 100

In [ ]:
data_train2 = pd.read_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train.csv')
data_train2.head()

,review_type,review_content,rev_list,rev_kalimat
21091,1,the biggest problem is the jumbled and hackney...,"[biggest, problem, jumbl, hackney, screenplay,...",biggest problem jumbl hackney screenplay regar...
18183,1,the film tries to cover the logic gaps with co...,"[film, cover, logic, gap, comedi, joke, bad, w...",film cover logic gap comedi joke bad worsen ex...
3795,0,this is a sublime romantic comedy that spins o...,"[sublim, romant, comedi, spin, wilder, favouri...",sublim romant comedi spin wilder favourit them...
22590,1,"unfortunately, it all builds to a last reel th...","[build, reel, rail, strenuou, bit, action, fli...",build reel rail strenuou bit action flick sill...
1280,0,"sombre and sensitive, hittman's film offers a ...","[sombr, sensit, hittman, film, offer, sear, ou...",sombr sensit hittman film offer sear outrag in...
...,...,...,...,...
11964,1,...an absolutely interminable piece of work...,"[absolut, intermin, piec, work]",absolut intermin piec work
21577,1,with a promising title taken from edgar allan ...,"[promis, titl, edgar, allan, poe, poem, haunt,...",promis titl edgar allan poe poem haunt palac m...
5390,0,amusing and unusual film about the bond betwee...,"[amus, unusu, film, bond, titl, charact]",amus unusu film bond titl charact
860,0,uncut gems is driven by a solid performance by...,"[uncut, gem, driven, solid, perform, adam, san...",uncut gem driven solid perform adam sandler en...


In [ ]:
print('Jumlah Data :', len(data_train2))
print('Sentiment : ', data_train2.groupby(['review_type'])['review_type'].count())

Jumlah Data : 16457
Sentiment :  review_type
0    8228
1    8229
Name: review_type, dtype: int64


In [ ]:
data_test2 = pd.read_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test.csv')
data_test2.head()

In [ ]:
print('Jumlah Data :', len(data_test2))
print('Sentiment : ', data_test2.groupby(['review_type'])['review_type'].count())

Jumlah Data : 7053
Sentiment :  review_type
0    3528
1    3525
Name: review_type, dtype: int64


##W2V Train

In [ ]:
Corpus_voc = [] 
temp = []
# iterate through each sentence in the file 
for i in dataword2vec: 
  temp = [] 
  # tokenize the sentence into words 
  for j in word_tokenize(str(i)): 
      temp.append(j.lower()) 
  Corpus_voc.append(temp)

In [ ]:
data_train2['rev_list'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         data_train2.rev_list))  # tokenize sentences

In [ ]:
modelskip2 = Word2Vec(Corpus_voc, min_count = 1 , alpha = 0.1 , size=100, window=5, sg=1)

In [ ]:
joblib.dump(modelskip2, '/content/drive/My Drive/TUGAS AKHIR/TA/Code/word2vecskip.jl')
skipgram = joblib.load('/content/drive/My Drive/TUGAS AKHIR/TA/Code/word2vecskip.jl')

In [ ]:
def get_w2v_features(w2v_model, sentence_group):
    words = np.concatenate(sentence_group)  # words in text
    index2word_set = set(w2v_model.wv.vocab.keys())  # words known to model
    
    featureVec = np.zeros(w2v_model.vector_size, dtype="float32")
    
    # Initialize a counter for number of words in a review
    nwords = 0
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

#mengambil arti dari semua vektor kata yang ada pada kalimat untuk mendapatkan representasi vektor 

In [ ]:
# memberi nomor index
index_kosong = []
for i in range(len(data_train2.rev_list)):
  if len(data_train2.rev_list.iloc[i]) == 0:
    index_kosong.append(i)

In [ ]:
# drop index yang masih kosong 
data_train2 = data_train2.drop(data_train2.index[index_kosong])

In [ ]:
data_train2['w2v_features_skip'] = list(map(lambda sen_group:
                                      get_w2v_features(modelskip2, sen_group),
                                      data_train2.rev_list))

In [ ]:
feature = [x for x in data_train2['w2v_features_skip'].transpose()]
word2vec_with_stop_features_train2 = np.asarray(feature)

print(word2vec_with_stop_features_train2.shape)

(16441, 100)


In [ ]:
X_train2 = word2vec_with_stop_features_train2

##W2V Test

In [ ]:
data_test2['rev_list'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         data_test2.rev_list))  # tokenize sentences

In [ ]:
# memberi nomor index
index_kosong = []
for i in range(len(data_test2.rev_list)):
  if len(data_test2.rev_list.iloc[i]) == 0:
    index_kosong.append(i)

In [ ]:
# drop index yang masih kosong 
data_test2 = data_test2.drop(data_test2.index[index_kosong])

In [ ]:
data_test2['w2v_features_skip'] = list(map(lambda sen_group:
                                      get_w2v_features(modelskip2, sen_group),
                                      data_test2.rev_list))

In [ ]:
feature = [x for x in data_test2['w2v_features_skip'].transpose()]
word2vec_with_stop_features_test2 = np.asarray(feature)

print(word2vec_with_stop_features_test2.shape)

(7048, 100)


In [ ]:
X_test2 = word2vec_with_stop_features_test2

##Gaussian

In [ ]:
y_train2 = data_train2['review_type']
y_test2 = data_test2['review_type']

In [ ]:
model = GaussianNB()
model.fit(X_train2, y_train2)
pred2 = model.predict(X_train2) #prediksi train
Y_pred2 = model.predict(X_test2) #prediksi test
Y_pred2

array([0, 1, 1, ..., 0, 0, 0])

### Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test2, Y_pred2)*100, "%")
print("Precision: ", precision_score(y_test2, Y_pred2)*100, "%")
print("Recall: ", recall_score(y_test2, Y_pred2)*100, "%")
print("F1 Score: ", f1_score(y_test2, Y_pred2)*100, "%")

Accuracy: 63.138479001135074 %
Precision:  66.42984014209591 %
Recall:  53.07976156684644 %
F1 Score:  59.0091511517829 %


In [ ]:
print("Confusion Matrix")
print(confusion_matrix(y_test2, Y_pred2))
tn, fp, fn, tp = confusion_matrix(y_test2, Y_pred2).ravel()
print('TN: ', tn)
print('FP: ', fp)
print('FN: ', fn)
print('TP: ', tp)

Confusion Matrix
[[2580  945]
 [1653 1870]]
TN:  2580
FP:  945
FN:  1653
TP:  1870


In [ ]:
print("Classification report")
print(classification_report(y_test2, Y_pred2))

Classification report
              precision    recall  f1-score   support

           0       0.61      0.73      0.67      3525
           1       0.66      0.53      0.59      3523

    accuracy                           0.63      7048
   macro avg       0.64      0.63      0.63      7048
weighted avg       0.64      0.63      0.63      7048



### Export data

In [ ]:
data_train_ex2 = data_train2.assign(sentiment_pred = pred2)

In [ ]:
data_train_ex2.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_gaussian_100_pred.csv')

In [ ]:
data_test_ex2 = data_test2.assign(sentiment_pred = Y_pred2)

In [ ]:
data_test_ex2.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_gaussian_100_pred.csv')

##Multinomial

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train2_multi = scaler.fit_transform(X_train2)
X_test2_multi = scaler.fit_transform(X_test2)

In [ ]:
model2_multi = MultinomialNB()
model2_multi.fit(X_train2_multi, y_train2)
pred2_multi = model2_multi.predict(X_train2_multi) #prediksi train
Y_pred2_multi = model2_multi.predict(X_test2_multi) #prediksi test
Y_pred2_multi

array([0, 1, 1, ..., 0, 1, 0])

### Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test2, Y_pred2_multi)*100, "%")
print("Precision: ", precision_score(y_test2, Y_pred2_multi)*100, "%")
print("Recall: ", recall_score(y_test2, Y_pred2_multi)*100, "%")
print("F1 Score: ", f1_score(y_test2, Y_pred2_multi)*100, "%")

Accuracy: 65.13904653802497 %
Precision:  72.52747252747253 %
Recall:  48.708487084870846 %
F1 Score:  58.27814569536424 %


In [ ]:
print("Confusion Matrix")
print(confusion_matrix(y_test2, Y_pred2_multi))
tn, fp, fn, tp = confusion_matrix(y_test2, Y_pred2_multi).ravel()
print('TN: ', tn)
print('FP: ', fp)
print('FN: ', fn)
print('TP: ', tp)

Confusion Matrix
[[2875  650]
 [1807 1716]]
TN:  2875
FP:  650
FN:  1807
TP:  1716


In [ ]:
print("Classification report")
print(classification_report(y_test2, Y_pred2_multi))

Classification report
              precision    recall  f1-score   support

           0       0.61      0.82      0.70      3525
           1       0.73      0.49      0.58      3523

    accuracy                           0.65      7048
   macro avg       0.67      0.65      0.64      7048
weighted avg       0.67      0.65      0.64      7048



### Export data

In [ ]:
data_train_ex2_multi = data_train2.assign(sentiment_pred = pred2_multi)

In [ ]:
data_train_ex2_multi.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_multinomial_100_pred.csv')

In [ ]:
data_test_ex2_multi = data_test2.assign(sentiment_pred = Y_pred2_multi)

In [ ]:
data_test_ex2_multi.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_multinomial_100_pred.csv')